In [ ]:
import pandas as pd

In [ ]:
# Specify the directory path as a string
directory_path: str = 'data'

#index/time/divice/mac_id/rssi/sward_id/zone_id
df: pd.DataFrame = pd.read_csv('data/tjlabs_sward.csv')


df.columns = ['index', 'time', 'device', 'mac_id', 'rssi', 'sward_id', 'zone_id']

df.head()

In [ ]:
df['time'] = pd.to_datetime(df['time']) 

df.describe()

In [ ]:
mac_counts = df.groupby('mac_id').size()

# Convert the Series to a dictionary (optional)
mac_counts_dict = mac_counts.to_dict()
mac_counts_dict.__len__()

In [ ]:
df= df.sort_values(by=['time'])
df

In [ ]:
grouped_df = df.groupby('time')
grouped_df.count()

In [ ]:
from circular_queue import CircularQueue
import uuid
import pandas as pd

# Initialize the circular queue and dictionaries
queue = CircularQueue(5)
temp_mac_dict = {}
main_mac_dict = {}

# Preprocessing step: Create a dictionary for last occurrences
last_occurrences = df.groupby('mac_id').agg({'zone_id': 'last', 'rssi': 'last', 'time': 'last'}).to_dict('index')

# Function to process new MAC IDs
def process_new_mac_ids(new_mac_ids):
    for mac_id in new_mac_ids:
        if mac_id not in temp_mac_dict:
            temp_mac_dict[mac_id] = uuid.uuid4()

# Function to process disappeared MAC IDs
def process_disappeared_mac_ids(disappeared_mac_ids, one_epoch_ago, two_epoch_ago, last_updated_time):
    for disappeared_mac_id in disappeared_mac_ids:
        last_info = last_occurrences.get(disappeared_mac_id)
        if last_info and last_info['time'] == last_updated_time:
            last_zone_id = last_info['zone_id']
            for mac_id in one_epoch_ago.union(two_epoch_ago):
                current_info = last_occurrences.get(mac_id)
                if current_info and current_info['zone_id'] == last_zone_id:
                    if abs(current_info['rssi'] - last_info['rssi']) < 10 : 
                      main_mac_dict[mac_id] = temp_mac_dict[disappeared_mac_id]

# Main processing loop
for _, group in zip(range(16850), grouped_df):
    time, temp_df = group[0], group[1]
    unique_mac_ids = temp_df['mac_id'].unique()

    queue.enqueue([time, unique_mac_ids])
    print(queue.last_updated)

    if queue.new_mac_set:
        process_new_mac_ids(queue.new_mac_set)

    if queue.disappeared_mac_set:
        one_epoch_ago = queue.new_mac_queue[(queue.tail - 2) % queue.capacity]
        two_epoch_ago = queue.new_mac_queue[(queue.tail - 3) % queue.capacity]
        last_updated_time = queue.last_updated_queue[(queue.tail - 2) % queue.capacity]

        process_disappeared_mac_ids(queue.disappeared_mac_set, one_epoch_ago, two_epoch_ago, last_updated_time)

In [ ]:
temp_mac_dict

In [ ]:
main_mac_dict

In [ ]:
# from circular_queue import CircularQueue
# import uuid
# import pandas as pd

# # Initialize the circular queue and dictionaries
# queue = CircularQueue(6)
# temp_mac_dict = {}
# main_mac_dict = {}

# # Preprocessing step: Create a dictionary for last occurrences
# last_occurrences = df.groupby('mac_id').agg({'zone_id': 'last', 'rssi': 'last', 'time': 'last'}).to_dict('index')
# grouped_by_mac = df.groupby('mac_id')

# def filter_by_custom_time(group, cutoff):
#     filtered_group = group[group['time'] <= cutoff]
#     return filtered_group.loc[filtered_group['time'].idxmax()] if not filtered_group.empty else None

# # Function to process new MAC IDs
# def process_new_mac_ids(new_mac_ids):
#     for mac_id in new_mac_ids:
#         if mac_id not in temp_mac_dict:
#             temp_mac_dict[mac_id] = uuid.uuid4()

# # Function to process disappeared MAC IDs
# def process_disappeared_mac_ids(disappeared_mac_ids, one_epoch_ago, two_epoch_ago, last_updated_time):
#     last_occurrences = grouped_by_mac.apply(filter_by_custom_time, cutoff=last_updated_time)
#     for disappeared_mac_id in disappeared_mac_ids:
#         last_info = last_occurrences.get(disappeared_mac_id)
#         if last_info and last_info['time'] == last_updated_time:
#             last_zone_id = last_info['zone_id']
#             for mac_id in one_epoch_ago.union(two_epoch_ago):
#                 current_info = last_occurrences.get(mac_id)
#                 if current_info and current_info['zone_id'] == last_zone_id:
#                     main_mac_dict[mac_id] = temp_mac_dict[disappeared_mac_id]

# # Main processing loop
# for _, group in zip(range(1000), grouped_df):
#     time, temp_df = group[0], group[1]
#     unique_mac_ids = temp_df['mac_id'].unique()

#     queue.enqueue([time, unique_mac_ids])
#     print(queue.last_updated)

#     if queue.new_mac_set:
#         process_new_mac_ids(queue.new_mac_set)

#     if queue.disappeared_mac_set:
#         one_epoch_ago = queue.new_mac_queue[(queue.tail - 2) % queue.capacity]
#         two_epoch_ago = queue.new_mac_queue[(queue.tail - 3) % queue.capacity]
#         last_updated_time = queue.last_updated_queue[(queue.tail - 2) % queue.capacity]

#         process_disappeared_mac_ids(queue.disappeared_mac_set, one_epoch_ago, two_epoch_ago, last_updated_time)

In [ ]:
# temp_mac_dict

In [ ]:
# main_mac_dict

In [ ]:
# df[df['mac_id'] == 'F5-4F-73-13-A7-BB']